#### 拉取订单

In [ ]:
import pymysql
import pandas as pd
import numpy as np
from functools import reduce

In [ ]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [ ]:
dict = pd.read_excel(r"C:\Users\13670\Desktop\dict.xlsx")
id_list = dict.loc[:,'TagID'].values.tolist()

In [ ]:
tag_df = pd.DataFrame()

for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df

In [ ]:
tag_df.to_pickle(r"C:\Users\13670\Desktop\fei.pkl")

#### 关联分析

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np

In [ ]:
def PrepareData (tagall):
    ordercode_list = []
    taglist = []
    for order_code in np.unique(tagall.ORDER_CODE):
        taglist_ = tagall.loc[tagall.ORDER_CODE == order_code, 'TagName'].values.tolist() 
        ordercode_list.append(order_code)
        taglist.append(taglist_)
    df = pd.DataFrame({"order_code": ordercode_list, "taglist":taglist})
    return df

In [ ]:
def AprioriAnalysis(dfData,min_supp,min_conf):
    DataList = dfData.taglist.values.tolist()
    te = TransactionEncoder()
    df_tf = te.fit_transform(DataList)
    df = pd.DataFrame(df_tf,columns=te.columns_)

    frequent_itemsets = apriori(df,min_support= min_supp,use_colnames= True)
    rules = association_rules(frequent_itemsets,metric = 'confidence',min_threshold = min_conf)
    # rules = rules.drop(rules[rules.lift <1.0].index)
    rules.rename(columns = {'antecedents':'from','consequents':'to','support':'sup','confidence':'conf'},inplace = True)
    rules.sort_values(by = ['sup','conf'],ascending = (False,False))
    return rules

In [ ]:
renal = pd.read_pickle (r'C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\tagall.pkl')

In [ ]:
renal_dict =  pd.read_excel (r'C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renal_dict.xlsx')

In [ ]:
renal_ = renal.copy()

In [ ]:
renal_list_AB = renal_dict.loc[renal_dict.logic1.isin (["A",'a','B','b']),["tag_id",'TagName']]
renal_list_A = renal_dict.loc[renal_dict.logic1.isin (["A",'a']),["tag_id",'TagName']]
renal_list_B = renal_dict.loc[renal_dict.logic1.isin (["B",'b']),["tag_id",'TagName']]

In [ ]:
renal_AB = pd.merge(renal_,renal_list_AB, left_on = 'TAG_ID', right_on = 'tag_id').drop(columns = ['TAG_ID','tag_id'])
renal_A = pd.merge(renal_,renal_list_A, left_on = 'TAG_ID', right_on = 'tag_id').drop(columns = ['TAG_ID','tag_id'])
renal_B = pd.merge(renal_,renal_list_B, left_on = 'TAG_ID', right_on = 'tag_id').drop(columns = ['TAG_ID','tag_id'])

In [ ]:
df_AB = PrepareData(renal_AB)

In [ ]:
rules_AB = AprioriAnalysis(df_AB,0.001,0.05)

#### 穷举订单

#### 1. 描述——结论

In [ ]:
from itertools import product

In [ ]:
tagfull = pd.read_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\tagfull.pkl")
renal = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201016\renal_dict.xlsx")

In [ ]:
desclist = renal.loc[renal['logic']== "A",'abbr'].drop_duplicates()
summlist = renal.loc[renal['logic']=="a",'abbr'].drop_duplicates()

In [ ]:
renal_dict = renal.set_index("TAG_ID").to_dict()["abbr"]
tagfull.replace(renal_dict,inplace = True)

In [ ]:
DescDf = tagfull.loc[tagfull['TAG_ID'].isin(desclist)].drop_duplicates()
SummDf = tagfull.loc[tagfull['TAG_ID'].isin(summlist)].drop_duplicates()

In [ ]:
def ExhaustPatterns(DescDf,SummDf):
    ordercode=[]
    desclist=[]
    summlist=[]
    for item in DescDf['ORDER_CODE'].drop_duplicates():
        desclistsub=DescDf.loc[DescDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        summlistsub=SummDf.loc[SummDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        ordercode.append(item)
        desclist.append(desclistsub)
        summlist.append(summlistsub)
    TagOrderCode=pd.DataFrame({'ORDER_CODE':ordercode,'desclist':desclist,'summlist':summlist}) 
    return TagOrderCode

TagOrderCode = ExhaustPatterns(DescDf,SummDf)
TagOrderCode

In [ ]:
TagOrderCode.to_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\TagOrderCode.pkl")

In [ ]:
df = TagOrderCode.applymap(lambda x: tuple(x))
df_tuple = pd.DataFrame(df.groupby(['desclist','summlist'])['ORDER_CODE'].count()).reset_index()
df_tuple.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\C1.xlsx")

In [ ]:
summlist2 = list(summlist)
summlist2.insert(-1,"missing")

In [ ]:
from itertools import product 

summ = []
desc = []
for n in product(desclist,summlist2):
    summsub = n[1]
    descsub = n[0]
    summ.append(summsub)
    desc.append(descsub)
    
solid = pd.DataFrame({'desc':desc,'summ':summ})

In [ ]:
solid.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\solid.xlsx")
# 填写solid，命名为solid列

In [ ]:
def Count_and_Disassemble(TagOrderCode,solid_path):
    df = TagOrderCode.applymap(lambda x: tuple(x))
    df_tuple = pd.DataFrame(df.groupby(['desclist','summlist'])['ORDER_CODE'].count()).reset_index()
    solid = pd.read_excel(solid_path)
    
    desc_list = []
    summ_list = []
    count_list = []
    source_model = []
    S_NO_desc = []
    S_NO_summ = []
    S_NO_count = []

    for line in df_tuple.values:
#         print(line)
#         print(line[0])
#         print(line[1])
        desc = eval(str(line[0]))
        summ = eval(str(line[1]))
        count = line[2]  

        if len(summ)==0:
            source = "missing"
            for index in range(len(desc)):
                descsub = desc[index]
                desc_list.append(descsub)
                summ_list.append(0)
                count_list.append(count)
                source_model.append(source)

        elif (len(desc)== 1 & len(summ)==1):
            source = "complete"
            desc_list.append(desc[0])
            summ_list.append(summ[0])
            count_list.append(count)
            source_model.append(source)
        else :
            source = "split"
            descsub_list = []
            summsub_list = []

            for n in product(desc,summ):
                dfsub = pd.DataFrame(product(desc,summ), columns = ['desc','summ'])
                dfsub_solid_merge = pd.merge(dfsub,solid,how = 'left')
                df_new = dfsub_solid_merge.loc[dfsub_solid_merge.solid.values != 0,["desc","summ"]]
            if dfsub_solid_merge.solid.values.sum()== 0 :
                    S_NO_desc.append(desc)
                    S_NO_summ.append(summ)
                    S_NO_count.append(count)                      
            for index_n, item in enumerate (np.unique(df_new.desc)):
                countsub = count/len(np.unique(df_new.desc))
                summsublist = df_new.loc[df_new["desc"].isin ([item]),'summ']
                for summsub_ in summsublist:
                    countsub_ = countsub/len(summsublist)
                    desc_list.append(item)
                    summ_list.append(summsub_)
                    count_list.append(countsub_)
                    source_model.append(source)
    final = pd.DataFrame({"desc":desc_list,"summ":summ_list,"count":count_list,"source_model":source_model}) 
    NO_ = pd.DataFrame({"desc":S_NO_desc,"summ":S_NO_summ,"count":S_NO_count})
    return final,NO_

final,NO = Count_and_Disassemble(TagOrderCode,r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\solid.xlsx")
final

In [ ]:
solid = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\solid.xlsx",index_col= False )
solid = solid.iloc[:,1:]

In [ ]:
final1 = final.copy()
final1.loc[final1["summ"] == 0,"summ"] ="missing"
final_ = pd.merge(final1,solid,left_on =[ "desc" ,"summ"] ,right_on = ["desc","summ"])
final_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\final_.xlsx")
NO.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\NO.xlsx")
class_dict = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\solid.xlsx",sheet_name = "Sheet2")
final_solid = pd.DataFrame(final_.groupby(['desc','summ','source_model',"solid"])["count"].sum())
final_solid.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\final_solid.xlsx")
final_solid1 = final_solid.copy()
final_solid1 = final_solid1.reset_index(['desc','summ','source_model','solid'])
class_df = pd.merge(final_solid1,class_dict,left_on = ["source_model","solid"],right_on = ["source_model","solid"])
pic_df = class_df.loc[:,["desc","summ","count","result"]]
pic_df_ = pd.DataFrame(pic_df.groupby(["desc","summ","result"])["count"].sum())
pic_df_ = pic_df_.reset_index(['desc','summ','result'])
pic_df_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\pic.xlsx")

pic_df_1 = pic_df_.loc[pic_df_["result"] != "NO",:] # label为NO的订单不计算占比
calu = pd.pivot_table(pic_df_1,index = "desc",columns = "summ",values = "count",aggfunc = np.sum)
calu["sum"] = calu.sum(axis = 1)
picdf1 = pd.DataFrame(pic_df_.groupby(["desc",'summ'])["count"].sum()).reset_index(["desc",'summ'])
sum1 = pd.merge(picdf1,calu,left_on =[ "desc"] ,right_on = ["desc"])
sum2 = sum1.loc[:,['desc','summ','count','sum']]
sum2['count_per'] = sum2['count']/sum2['sum']*100
sum3 = pd.merge(pic_df_,sum2,left_on =["desc","summ"] ,right_on = ["desc","summ"])
sum4= sum3.loc[sum3['result'] == "Maybe",:]
sum4["Maybe_per"] = sum4['count_x']/sum4['count_y']*100
sum4 = sum4.loc[:,['desc','summ','Maybe_per']]
sum5 = pd.merge(sum2,sum4,left_on =["desc","summ"] ,right_on = ["desc","summ"],how = "outer")
nodf = pic_df_.loc[pic_df_['result'] == "NO",['desc','summ','result']]
sum6 = pd.merge(sum5,nodf,left_on =["desc","summ"] ,right_on = ["desc","summ"],how = "outer")
sum6.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201219\summary.xlsx")

#### 画图

In [ ]:
import pandas as pd
from pyecharts.charts import Sankey
import pyecharts.options as opts

picdf = pic_df_.loc[pic_df_.result != "NO",['desc','summ','count']]

nodes = []

for i in range(2):
    values = picdf.iloc[:,i].unique()
    for value in values:
        dict_nodes = {}
        dict_nodes["name"] = value
        nodes.append(dict_nodes)
        
links = []
 
for j in picdf.values:
    dict_links = {}
    dict_links["source"]= j[0]
    dict_links["target"]= j[1]
    dict_links["value"]= j[2]
#     dict_links["result"]= j[3]
    links.append(dict_links)
    
pic = (
    Sankey()
    .add('回声分析(数量)', 
         nodes,    
         links,
         itemstyle_opts=opts.ItemStyleOpts(border_width=4, border_color="#aaaa"),
         linestyle_opt=opts.LineStyleOpts(color="source", curve=0.1, opacity=0.4),
         tooltip_opts=opts.TooltipOpts(trigger_on="mousemove"),
         label_opts=opts.LabelOpts(position="right"),
         node_gap = 3,
    )
    .set_global_opts(title_opts=opts.TitleOpts(title = '肾脏局部病变回声分析'))
)
 
pic.render_notebook()

#### 结论——描述

In [ ]:
def ExhaustPatterns(DescDf,SummDf):
    ordercode=[]
    desclist=[]
    summlist=[]
    for item in SummDf['ORDER_CODE'].drop_duplicates():
        desclistsub=DescDf.loc[DescDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        summlistsub=SummDf.loc[SummDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        ordercode.append(item)
        desclist.append(desclistsub)
        summlist.append(summlistsub)
    TagOrderCode=pd.DataFrame({'ORDER_CODE':ordercode,'summlist':summlist,'desclist':desclist}) 
    return TagOrderCode

TagOrderCode1 = ExhaustPatterns(DescDf,SummDf)
TagOrderCode1

In [ ]:
df = TagOrderCode1.applymap(lambda x: tuple(x))
df_tuple = pd.DataFrame(df.groupby(['summlist','desclist'])['ORDER_CODE'].count()).reset_index()
df_tuple.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\C1.xlsx")

In [ ]:
from itertools import product 

desclist2 = list(desclist)
desclist2.insert(-1,"missing")

summ = []
desc = []
for n in product(desclist2,summlist):
    summsub = n[1]
    descsub = n[0]
    summ.append(summsub)
    desc.append(descsub)
    
solid = pd.DataFrame({'summ':summ,'desc':desc})

In [ ]:
solid.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\solid.xlsx")

In [ ]:
def Count_and_Disassemble(TagOrderCode,solid_path):
    df = TagOrderCode.applymap(lambda x: tuple(x))
    df_tuple = pd.DataFrame(df.groupby(['summlist','desclist'])['ORDER_CODE'].count()).reset_index()
    solid = pd.read_excel(solid_path)
    
    desc_list = []
    summ_list = []
    count_list = []
    source_model = []
    S_NO_desc = []
    S_NO_summ = []
    S_NO_count = []

    for line in df_tuple.values:
#         print(line)
#         print(line[0])
#         print(line[1])
        summ = eval(str(line[1]))
        desc = eval(str(line[0]))
        count = line[2]  

        if len(desc)==0:
            source = "missing"
            for index in range(len(summ)):
                summsub = summ[index]
                summ_list.append(summsub)
                desc_list.append(0)
                count_list.append(count)
                source_model.append(source)

        elif (len(summ)== 1 & len(desc)==1):
            source = "complete"
            desc_list.append(desc[0])
            summ_list.append(summ[0])
            count_list.append(count)
            source_model.append(source)
        else :
            source = "split"
            descsub_list = []
            summsub_list = []

            for n in product(summ,desc):
                dfsub = pd.DataFrame(product(summ,desc), columns = ['summ','desc'])
                dfsub_solid_merge = pd.merge(dfsub,solid,how = 'left')
                df_new = dfsub_solid_merge.loc[dfsub_solid_merge.solid.values != 0,["summ","desc"]]
            if dfsub_solid_merge.solid.values.sum()== 0 :
                    S_NO_desc.append(desc)
                    S_NO_summ.append(summ)
                    S_NO_count.append(count)                      
            for index_n, item in enumerate (np.unique(df_new.summ)):
                countsub = count/len(np.unique(df_new.summ))
                descsublist = df_new.loc[df_new["summ"].isin ([item]),'desc']
                for descsub_ in descsublist:
                    countsub_ = countsub/len(descsublist)
                    summ_list.append(item)
                    desc_list.append(descsub_)
                    count_list.append(countsub_)
                    source_model.append(source)
    final = pd.DataFrame({"summ":summ_list,"desc":desc_list,"count":count_list,"source_model":source_model}) 
    NO_ = pd.DataFrame({"summ":S_NO_summ,"desc":S_NO_desc,"count":S_NO_count})
    return final,NO_

final,NO = Count_and_Disassemble(TagOrderCode1,r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201217\solid.xlsx")

In [ ]:
final1 = final.copy()
final1.loc[final1["desc"] == 0,"desc"] ="missing"
final_ = pd.merge(final1,solid,left_on =[ "desc" ,"summ"] ,right_on = ["desc","summ"])
final_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201218\final_.xlsx")
NO.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201218\NO.xlsx")
class_dict = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201202\solid.xlsx",sheet_name = "Sheet2")
final_solid = pd.DataFrame(final_.groupby(['summ','desc','source_model',"solid"])["count"].sum())
final_solid.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201218\final_solid.xlsx")
final_solid1 = final_solid.copy()
final_solid1 = final_solid1.reset_index(['summ','desc','source_model','solid'])
class_df = pd.merge(final_solid1,class_dict,left_on = ["source_model","solid"],right_on = ["source_model","solid"])
pic_df = class_df.loc[:,["desc","summ","count","result"]]
pic_df_ = pd.DataFrame(pic_df.groupby(["summ","desc","result"])["count"].sum())
pic_df_ = pic_df_.reset_index(['summ','desc','result'])
pic_df_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201218\pic.xlsx")

pic_df_1 = pic_df_.loc[pic_df_["result"] != "NO",:]
calu = pd.pivot_table(pic_df_1,index = "summ",columns = "desc",values = "count",aggfunc = np.sum)
calu["sum"] = calu.sum(axis = 1)
picdf1 = pd.DataFrame(pic_df_.groupby(['summ',"desc"])["count"].sum()).reset_index(['summ',"desc"])
sum1 = pd.merge(picdf1,calu,left_on =["summ"] ,right_on = ["summ"])
sum2 = sum1.loc[:,['desc','summ','count','sum']]
sum2['count_per'] = sum2['count']/sum2['sum']*100
sum3 = pd.merge(pic_df_,sum2,left_on =["desc","summ"] ,right_on = ["desc","summ"])
sum4= sum3.loc[sum3['result'] == "Maybe",:]
sum4["Maybe_per"] = sum4['count_x']/sum4['count_y']*100
sum4 = sum4.loc[:,['desc','summ','Maybe_per']]
sum5 = pd.merge(sum2,sum4,left_on =["desc","summ"] ,right_on = ["desc","summ"],how = "outer")
nodf = pic_df_.loc[pic_df_['result'] == "NO",['desc','summ','result']]
sum6 = pd.merge(sum5,nodf,left_on =["desc","summ"] ,right_on = ["desc","summ"],how = "outer")
sum6.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201218\summary.xlsx")

#### 矩阵数据 & 热力图

In [ ]:
# data为将两次结果中的名称放到一起，作为矩阵的边
data = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201219\test.xlsx",sheet_name = "Sheet2")

In [ ]:
import seaborn as sns 
import itertools 
import matplotlib.pyplot as plt

results = pd.DataFrame(index=[i for i in np.unique(data.desc)],\
        columns=[i for i in np.unique(data.summ)])

for desc,summ in itertools.product(np.unique(data.desc),\
                               np.unique(data.summ)):
    data1 = data.loc[data.desc== desc,:]
    if desc == summ:
        perc = [0]
        results.loc[desc, summ] = 0
    else:
        perc = data1.loc[data1.summ== summ,"count_per"].values
        if len(perc):
            results.loc[desc, summ] = round(perc[0],2)
        else:
            continue
        
results

In [ ]:
results.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201219\matrix1.xlsx")

In [ ]:
results2_ = results[results.columns].astype(float)

fig, ax = plt.subplots(figsize=(15, 12))
ax = sns.heatmap(results2_,
                 annot=True,
             annot_kws={'size':12,'weight':'bold', 'color':'black'},
                 vmax=100, 
                 square=True, 
                 cmap="Blues",
                 center= 20
                )

ax.set_title('Renal_Ultra_Sound')
ax.set_xlabel('TO')
ax.set_ylabel("FROM")
# plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus']=False

plt.show()

#### graph

In [ ]:
results1 = results.fillna(float('inf'))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class Graph_Matrix:
    """
    Adjacency Matrix
    """
    def __init__(self, vertices=[], matrix=[]):
        """
        :param vertices:a dict with vertex id and index of matrix , such as {vertex:index}
        :param matrix: a matrix
        """
        self.matrix = matrix
        self.edges_dict = {}  # {(tail, head):weight}
        self.edges_array = []  # (tail, head, weight)
        self.vertices = vertices
        self.num_edges = 0

        # if provide adjacency matrix then create the edges list
        if len(matrix) > 0:
            if len(vertices) != len(matrix):
                raise IndexError
            self.edges = self.getAllEdges()
            self.num_edges = len(self.edges)

        # if do not provide a adjacency matrix, but provide the vertices list, build a matrix with 0
        elif len(vertices) > 0:
            self.matrix = [[0 for col in range(len(vertices))] for row in range(len(vertices))]

        self.num_vertices = len(self.matrix)

    def isOutRange(self, x):
        try:
            if x >= self.num_vertices or x <= 0:
                raise IndexError
        except IndexError:
            print("节点下标出界")

    def isEmpty(self):
        if self.num_vertices == 0:
            self.num_vertices = len(self.matrix)
        return self.num_vertices == 0

    def add_vertex(self, key):
        if key not in self.vertices:
            self.vertices[key] = len(self.vertices) + 1

        # add a vertex mean add a row and a column
        # add a column for every row
        for i in range(self.getVerticesNumbers()):
            self.matrix[i].append(0)

        self.num_vertices += 1

        nRow = [0] * self.num_vertices
        self.matrix.append(nRow)

    def getVertex(self, key):
        pass

    def add_edges_from_list(self, edges_list):  # edges_list : [(tail, head, weight),()]
        for i in range(len(edges_list)):
            self.add_edge(edges_list[i][0], edges_list[i][1], edges_list[i][2], )

    def add_edge(self, tail, head, cost=0):
        # if self.vertices.index(tail) >= 0:
        #   self.addVertex(tail)
        if tail not in self.vertices:
            self.add_vertex(tail)
        # if self.vertices.index(head) >= 0:
        #   self.addVertex(head)
        if head not in self.vertices:
            self.add_vertex(head)

        # for directory matrix
        self.matrix[self.vertices.index(tail)][self.vertices.index(head)] = cost
        # for non-directory matrix
        # self.matrix[self.vertices.index(fromV)][self.vertices.index(toV)] = \
        #   self.matrix[self.vertices.index(toV)][self.vertices.index(fromV)] = cost

        self.edges_dict[(tail, head)] = cost
        self.edges_array.append((tail, head, cost))
        self.num_edges = len(self.edges_dict)

    def getEdges(self, V):
        pass

    def getVerticesNumbers(self):
        if self.num_vertices == 0:
            self.num_vertices = len(self.matrix)
        return self.num_vertices

    def getAllVertices(self):
        return self.vertices

    def getAllEdges(self):
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix)):
                if 0 < self.matrix[i][j] < float('inf'):
                    self.edges_dict[self.vertices[i], self.vertices[j]] = self.matrix[i][j]
                    self.edges_array.append([self.vertices[i], self.vertices[j], self.matrix[i][j]])

        return self.edges_array

    def __repr__(self):
        return str(''.join(str(i) for i in self.matrix))

    def to_do_vertex(self, i):
        print('vertex: %s' % (self.vertices[i]))

    def to_do_edge(self, w, k):
        print('edge tail: %s, edge head: %s, weight: %s' % (self.vertices[w], self.vertices[k], str(self.matrix[w][k])))
        
my_graph = Graph_Matrix()     

In [ ]:
def draw_directed_graph(my_graph):
    G = nx.DiGraph()  # 建立一个空的无向图G
    for node in my_graph.vertices:
        G.add_node(str(node))
    G.add_weighted_edges_from(my_graph.edges_array)

    print("nodes:", G.nodes())  
    print("edges:", G.edges()) 
    print("number of edges:", G.number_of_edges())  
    
    plt.subplots(figsize=(8, 6))   
    nx.draw(G, with_labels=True,pos=nx.spring_layout(G),node_size = 500,
        width=[float(v['weight'])/100  for (r, c, v) in G.edges(data=True)])
    
    plt.rcParams['font.sans-serif'] = [u'SimHei']
    plt.rcParams['axes.unicode_minus']=False
    plt.savefig("directed_graph.png")
    plt.show()
    return G
    
def create_directed_matrix(my_graph,data):
    data1 = data.fillna(float('inf'))
    nodes = list(data.columns)
    matrix = data1.values
    my_graph = Graph_Matrix(nodes, matrix)
    return my_graph

if __name__ == '__main__':
    my_graph = Graph_Matrix()
    created_graph = create_directed_matrix(my_graph,results1_)
    G = draw_directed_graph(created_graph)

In [ ]:
# 生成邻近矩阵
a= nx.adjacency_matrix(G)
A=a.todense()

import numpy
numpy.savetxt('correlation_matrix.csv',A, delimiter = ',')


In [ ]:
def draw_directed_graph(my_graph):
    G = nx.DiGraph()  # 建立一个空的无向图G
    for node in my_graph.vertices:
        G.add_node(str(node))
    G.add_weighted_edges_from(my_graph.edges_array) 
    
    plt.subplots(figsize=(8, 6)) 
    
    elarge=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight']>20]
    esmall=[(u,v) for (u,v,d) in G.edges(data=True) if (d['weight']>0)&(d['weight']<=20)]
    nx.circular_layout(G)##图的布局方式，圆形
    pos=nx.spring_layout(G)
    nx.draw_networkx_nodes(G,pos,alpha=0.4,node_size = 600)
    #设置边的形式
    nx.draw_networkx_edges(G,pos,edgelist=elarge,width=3,alpha=1,edge_color='r')
    nx.draw_networkx_edges(G,pos,edgelist=esmall,width=1,alpha=0.8,edge_color='b',style='dashed')
    nx.draw_networkx_labels(G,pos,font_size=10)
    
    plt.rcParams['font.sans-serif'] = [u'SimHei']
    plt.rcParams['axes.unicode_minus']=False
    plt.savefig("directed_graph.png")
    plt.show()
    return G
    
def create_directed_matrix(my_graph,data):
    data1 = data.fillna(float('inf'))
    nodes = list(data.columns)
    matrix = data1.values
    my_graph = Graph_Matrix(nodes, matrix)
    return my_graph

if __name__ == '__main__':
    my_graph = Graph_Matrix()
    created_graph = create_directed_matrix(my_graph,results1_)
    G = draw_directed_graph(created_graph)

#### 提取异常订单

In [ ]:
Recode_df = pd.DataFrame()

for items in tag_df1.ORDER_CODE:
    ORDER_CODE_str = "'"+ items + "'"
    
    sqlstr = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
    FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
    AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000005D','S0000006D','S0000007D') \
    AND RR.ORDER_CODE IN (" + ORDER_CODE_str + ")AND DR.SZ_STATUS = 1 \
    LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
    AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
    WHERE \
    DR.ID IS NOT NULL \
    AND RR.SZ_STATUS = 1 \
    AND DR.SZ_STATUS = 1 \
    AND RR2.SZ_STATUS = 1 " 

    Recode = pd.read_sql(sqlstr,con=conn)
    Recode_df = pd.concat([Recode_df,Recode],axis = 0) 

Recode_df